In [1]:
!pip install telethon pillow tqdm

In [2]:
from telethon import TelegramClient
from telethon.tl.types import InputStickerSetShortName

from pathlib import Path
from tqdm import tqdm
from PIL import Image
import hashlib
import json
import random
import os


In [3]:
BASE_DIR = Path("../data")
RAW_DIR = BASE_DIR / "stickers_raw_tgs"     # raw downloaded stickers (webp)
PNG_DIR = BASE_DIR / "stickers_png"         # converted + resized PNG
UNIQUE_DIR = BASE_DIR / "stickers_unique"   # deduplicated
META_PATH = BASE_DIR / "metadata.json"

for d in [RAW_DIR, PNG_DIR, UNIQUE_DIR]:
    d.mkdir(exist_ok=True)

print("Directories:")
print(" RAW_DIR    =", RAW_DIR)
print(" PNG_DIR    =", PNG_DIR)
print(" UNIQUE_DIR =", UNIQUE_DIR)

Directories:
 RAW_DIR    = ../data/stickers_raw_tgs
 PNG_DIR    = ../data/stickers_png
 UNIQUE_DIR = ../data/stickers_unique


In [4]:
!pip install dotenv


In [5]:
from dotenv import load_dotenv
import os
load_dotenv()

api_id = os.getenv('TELEGRAM_APP_ID')   # int
api_hash = os.getenv('TELEGRAM_APP_HASH')  # str

# Session name (local file to store login)
SESSION_NAME = "stickergen_session"

In [6]:
# 👉 Add the short names of Telegram sticker packs you like
STICKER_PACKS = [
    "HotCherry",
    "Muffin",
    "NickWallowPig",
    "Muffin",
    "CaptainWhale",
    "BobJellyfish",
    "BananaFun",
    "VeryTiredPerson",
    "MrPugDog",
    "FroggoInLove"
]
print("Sticker packs to download:", STICKER_PACKS)


Sticker packs to download: ['HotCherry', 'Muffin', 'NickWallowPig', 'Muffin', 'CaptainWhale', 'BobJellyfish', 'BananaFun', 'VeryTiredPerson', 'MrPugDog', 'FroggoInLove']


In [7]:
from telethon.tl.functions.messages import GetStickerSetRequest
from telethon.tl.types import InputStickerSetShortName

async def download_sticker_pack(client, short_name):
    try:
        print(f"Downloading pack: {short_name}")
        
        # Request sticker set details
        sticker_set = await client(GetStickerSetRequest(
            stickerset=InputStickerSetShortName(short_name=short_name),
            hash=0
        ))
        
        count = 0
        for i, doc in enumerate(sticker_set.documents):
            out_path = RAW_DIR / f"{short_name}_{i}.tgs"
            if out_path.exists():
                continue
            
            await client.download_media(doc, file=out_path)
            count += 1
        
        print(f"Downloaded {count} stickers from {short_name}")
    except Exception as e:
        print(f"Failed to get sticker set {short_name}: {e}")
        return
    

async def main():
    for pack in STICKER_PACKS:
        await download_sticker_pack(client, pack)

client = TelegramClient(SESSION_NAME, api_id, api_hash)

async with client:
    for pack in STICKER_PACKS:
        await download_sticker_pack(client, pack)


Signed in successfully as Jiwon Hae; remember to not break the ToS or you will risk an account ban!
Downloaded 34 stickers from HotCherry
Downloaded 28 stickers from Muffin
Downloaded 31 stickers from NickWallowPig
Downloaded 0 stickers from Muffin
Downloaded 23 stickers from CaptainWhale
Downloaded 28 stickers from BobJellyfish
Downloaded 23 stickers from BananaFun
Downloaded 21 stickers from VeryTiredPerson
Downloaded 22 stickers from MrPugDog
Downloaded 27 stickers from FroggoInLove


In [16]:
import os
from pathlib import Path

from lottie.importers import importers
from lottie.exporters import exporters
from lottie.utils.stripper import float_strip, heavy_strip


def lottie_convert(
    infile: Path,
    out_dir: Path,
    fps=30,
    width=256,
    height=256,
    optimize=0,   # 0 = none, 1 = float strip, 2 = heavy strip
):
    infile = str(infile)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    
    # ------------------------
    # 1. Select importer
    # ------------------------
    ext = os.path.splitext(infile)[1][1:]
    importer = None
    for imp in importers:
        if ext in imp.extensions:
            importer = imp
            break

    if importer is None:
        raise ValueError(f"Unsupported extension .{ext}")

    animation = importer.process(infile)

    # ------------------------
    # 2. FPS update
    # ------------------------
    if fps:
        animation.frame_rate = fps

    # ------------------------
    # 3. Resize
    # ------------------------
    if width or height:
        if width and not height:
            height = animation.height * (width / animation.width)
        if height and not width:
            width = animation.width * (height / animation.height)
        animation.scale(width, height)

    # ------------------------
    # 4. Optimization
    # ------------------------
    if optimize == 1:
        float_strip(animation)
    elif optimize == 2:
        heavy_strip(animation)

    # ------------------------
    # 5. Export frames with correct template
    # ------------------------
    exporter = exporters.get("png")

    # python-lottie frame template:
    #   "{frame}"  → replaced with 0, 1, 2, ...
    outfile_template = str(out_dir / "frame.png")

    exporter.process(animation, outfile_template)

    return out_dir

RAW_DIR = Path("../data/stickers_raw_tgs")
PNG_DIR = Path("../data/stickers_png")

for tgs in RAW_DIR.glob("*.tgs"):
    output = PNG_DIR / tgs.stem
    lottie_convert(tgs, output)

In [ ]:
from rlottie_python import LottieAnimation
from PIL import Image
from pathlib import Path

def extract_frames_from_tgs(tgs_path: Path, out_dir: Path, size=256):
    out_dir.mkdir(parents=True, exist_ok=True)

    # Load Telegram TGS animation
    anim = LottieAnimation.from_tgs(str(tgs_path))
    total = anim.lottie_animation_get_totalframe()

    for i in range(total):
        frame = anim.render_pillow_frame(i)         # numpy array (H, W, 4)
        frame = frame.convert("RGBA").resize((size, size), Image.LANCZOS)
        frame.save(out_dir / f"frame_{i:04d}.png")

RAW_DIR = Path("../data/stickers_raw_tgs")
PNG_DIR = Path("../data/stickers_png")

for tgs in RAW_DIR.glob("*.tgs"):
    extract_frames_from_tgs(tgs, PNG_DIR / tgs.stem)

TypeError: expected string or buffer

In [20]:
from PIL import Image
from pathlib import Path

SRC_DIR = Path("../data/stickers_png")
OUT_DIR = Path("../data/icons_256")
OUT_DIR.mkdir(exist_ok=True)

TARGET_SIZE = 256

for sticker_dir in SRC_DIR.iterdir():
    if sticker_dir.is_dir():
        frames = sorted(sticker_dir.glob("*.png"))
        if not frames:
            continue

        first_frame = frames[0]
        img = Image.open(first_frame).convert("RGBA")
        img = img.resize((TARGET_SIZE, TARGET_SIZE), Image.LANCZOS)

        out_path = OUT_DIR / f"{sticker_dir.name}.png"
        img.save(out_path)
